In [1]:
!ls

Latex-Symbols.pdf
Project-2-细粒度情感分类.ipynb
comment-classification
weights_base.best.hdf5
情感分析2.py
情感分析notebook第四次调参.ipynb


In [2]:
import pandas as pd
import numpy as np

import jieba

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K

Using TensorFlow backend.


In [3]:
train = pd.read_csv('comment-classification/train.csv')
test = pd.read_csv('comment-classification/test.csv')
vaild = pd.read_csv('comment-classification/validationset.csv')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105000 entries, 0 to 104999
Data columns (total 22 columns):
id                                          105000 non-null int64
content                                     105000 non-null object
location_traffic_convenience                105000 non-null int64
location_distance_from_business_district    105000 non-null int64
location_easy_to_find                       105000 non-null int64
service_wait_time                           105000 non-null int64
service_waiters_attitude                    105000 non-null int64
service_parking_convenience                 105000 non-null int64
service_serving_speed                       105000 non-null int64
price_level                                 105000 non-null int64
price_cost_effective                        105000 non-null int64
price_discount                              105000 non-null int64
environment_decoration                      105000 non-null int64
environment_noise             

In [5]:
train.head(10)

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...",-2,-2,-2,-2,1,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,1,-2,1,-2
1,1,"""第三次参加大众点评网霸王餐的活动。这家店给人整体感觉一般。首先环境只能算中等，其次霸王餐提...",-2,-2,-2,-2,-2,-2,-2,0,...,0,0,0,0,1,-2,-2,-2,1,-2
2,2,"""4人同行 点了10个小吃\n榴莲酥 榴莲味道不足 松软 奶味浓\n虾饺 好吃 两颗大虾仁\...",-2,-2,-2,-2,0,-2,1,0,...,-2,-2,1,-2,0,1,-2,-2,0,-2
3,3,"""之前评价了莫名其妙被删 果断继续差评！ 换了菜单 价格更低 开始砸牌子 但套餐还是有150...",-2,-2,-2,-2,-2,-2,-2,0,...,-2,-2,-2,-2,-2,-1,-2,-2,-1,-1
4,4,"""出乎意料地惊艳，椰子鸡清热降火，美容养颜，大大满足了爱吃火锅怕上火星人。椰子冻是帅帅的老板...",-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,1,1,-2,1,-2
5,5,"""烤鸭使用的蘸酱很地道，觉得烤鸭好不好吃在很大程度上都取决于这个酱料是否正宗。个人感觉他们家...",-2,-2,-2,-2,1,-2,-2,0,...,-2,-2,1,-2,0,1,-2,-2,1,-2
6,6,"""在好友圈很红的原因很诡异.竟然基本都是我外地的朋友转发的.不远千里跑来厦门买回去吃.吃完各...",-2,-2,-1,-2,-2,-2,-2,-1,...,-2,-2,-1,-2,-1,1,1,1,1,-2
7,7,"""中午饭点人比较多需要排队～2点左右去吃的刚好可以用团购～也不用排队～就是平安夜店员说的都不...",-2,-2,-2,0,1,-2,-2,-2,...,-2,-2,-2,-1,-1,0,-2,1,1,1
8,8,"""跟朋友相约25号看港囧，首映当天排片率超高，为了提前订好票，选了文投国际影城，为了看电影我...",-2,1,-2,-2,-2,-2,-2,1,...,1,-2,-2,-2,-2,1,-2,-2,1,1
9,9,"""首先感谢大众点评再次抽中我免费品尝火锅，同时也感谢古蜀雒味三星堆主题火锅商家提供这个平台，...",-2,-2,1,-2,1,-2,-2,-2,...,1,1,1,1,0,-2,1,-2,0,1


In [6]:
def cut(string): return list(jieba.cut(string))

In [7]:
def handle_dummies(data):
    columns_names = data.columns.values[2:]

    concat_list = [data]

    for name in columns_names:
        print(name)
        concat_list.append(get_dummies(data, name))

    return pd.concat(concat_list, axis=1)

In [8]:
def get_dummies(data,name): return pd.get_dummies(data[name], prefix=name)

In [9]:
train =  handle_dummies(train)

test  =  handle_dummies(test)

location_traffic_convenience
location_distance_from_business_district
location_easy_to_find
service_wait_time
service_waiters_attitude
service_parking_convenience
service_serving_speed
price_level
price_cost_effective
price_discount
environment_decoration
environment_noise
environment_space
environment_cleaness
dish_portion
dish_taste
dish_look
dish_recommendation
others_overall_experience
others_willing_to_consume_again
location_traffic_convenience
location_distance_from_business_district
location_easy_to_find
service_wait_time
service_waiters_attitude
service_parking_convenience
service_serving_speed
price_level
price_cost_effective
price_discount
environment_decoration
environment_noise
environment_space
environment_cleaness
dish_portion
dish_taste
dish_look
dish_recommendation
others_overall_experience
others_willing_to_consume_again


In [10]:
all_contents = train['content'].tolist()
all_contents = [' '.join(cut(s)) for s in all_contents]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_3/nm3s5d4x7h199bbtc4z__fth0000gn/T/jieba.cache
Loading model cost 2.436 seconds.
Prefix dict has been built succesfully.


In [11]:
y = train[train.columns[-80:]].values
list_sentences_test = test['content'].fillna("CVxTz").values

In [13]:
max_features = 20000
maxlen = 100
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(all_contents)
list_tokenized_train = tokenizer.texts_to_sequences(all_contents)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [15]:

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def fmeasure(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=1)

In [18]:
def get_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Bidirectional(LSTM(32, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.05)(x)
    x = Dense(32, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(80, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy',fmeasure])

    return model



In [19]:
model = get_model()
batch_size = 64
epochs = 20

file_path="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=20)


callbacks_list = [checkpoint, early] #early
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

model.load_weights(file_path)

y_test = model.predict(X_te)

Train on 94500 samples, validate on 10500 samples
Epoch 1/20
94500/94500 [==============================] - 1810s 19ms/step - loss: 0.3526 - acc: 0.8490 - fmeasure: 0.6836 - val_loss: 0.3222 - val_acc: 0.8642 - val_fmeasure: 0.7143

Epoch 00001: val_loss improved from inf to 0.32225, saving model to weights_base.best.hdf5
Epoch 2/20
94500/94500 [==============================] - 1638s 17ms/step - loss: 0.3155 - acc: 0.8698 - fmeasure: 0.7258 - val_loss: 0.3073 - val_acc: 0.8758 - val_fmeasure: 0.7400

Epoch 00002: val_loss improved from 0.32225 to 0.30732, saving model to weights_base.best.hdf5
Epoch 3/20
94500/94500 [==============================] - 1629s 17ms/step - loss: 0.3033 - acc: 0.8768 - fmeasure: 0.7419 - val_loss: 0.3021 - val_acc: 0.8781 - val_fmeasure: 0.7450

Epoch 00003: val_loss improved from 0.30732 to 0.30211, saving model to weights_base.best.hdf5
Epoch 4/20
94500/94500 [==============================] - 1631s 17ms/step - loss: 0.2946 - acc: 0.8802 - fmeasure: 0.748

[    -2  -1   0   1
 0    0   1   0   0
 1    1   0   0   0
 2    0   1   0   0
 3    1   0   0   0
 4    1   0   0   0
 5    1   0   0   0
 6    1   0   0   0
 7    1   0   0   0
 8    0   0   1   0
 9    0   0   0   1
 10   0   1   0   0
 11   1   0   0   0
 12   1   0   0   0
 13   0   1   0   0
 14   1   0   0   0
 15   0   0   1   0
 16   1   0   0   0
 17   1   0   0   0
 18   0   0   1   0
 19   1   0   0   0,     -2   1
 0    0   1
 1    1   0
 2    1   0
 3    1   0
 4    0   1
 5    1   0
 6    0   1
 7    0   1
 8    0   1
 9    0   1
 10   0   1
 11   1   0
 12   1   0
 13   1   0
 14   1   0
 15   0   1
 16   0   1
 17   1   0
 18   0   1
 19   0   1,     -2   1
 0    0   1
 1    1   0
 2    1   0
 3    1   0
 4    1   0
 5    1   0
 6    1   0
 7    0   1
 8    0   1
 9    1   0
 10   0   1
 11   0   1
 12   0   1
 13   0   1
 14   0   1
 15   0   1
 16   0   1
 17   0   1
 18   0   1
 19   1   0,     -2  -1   1
 0    1   0   0
 1    1   0   0
 2    1   0   0
 3    1   0 

array([[-1, -2, -1, ..., -2,  0, -2],
       [ 1, -2, -2, ..., -2,  1,  1],
       [ 1, -2, -2, ...,  1,  1, -2],
       ...,
       [-2, -2, -2, ..., -2,  1, -2],
       [-2, -2,  1, ..., -2,  0, -1],
       [-2,  1, -2, ..., -2,  1, -2]])

In [23]:
list_sentences_test = test['content'].fillna("CVxTz").values

NameError: name 'ModelCheckpoint' is not defined